In [25]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomNormal

In [7]:
df_main = pd.read_csv('higgs_million_sample.csv')

In [8]:
df_main

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.0,0.333077,1.165787,1.450824,0.606966,0.341805,1.230193,-0.208964,-0.611420,2.173076,...,-1.639457,0.615236,0.000000,0.571461,0.859411,0.981772,0.671796,1.486963,0.848153,0.709969
1,1.0,0.666885,0.400247,0.659037,1.068549,-0.017721,0.994580,-0.220847,-1.593219,2.173076,...,0.177763,-0.173952,0.000000,0.790915,1.085007,0.981199,0.910133,0.870022,0.874154,0.773464
2,1.0,0.703670,-0.690598,-0.007454,1.420747,0.490262,0.969571,0.688184,-1.425798,0.000000,...,-0.983192,0.700138,3.101961,0.774349,1.110985,0.995739,0.515127,0.872392,0.901602,1.189014
3,1.0,0.777240,-1.094795,0.360522,0.813867,-0.437939,0.788738,-0.357498,0.465836,1.086538,...,1.905039,-0.622878,0.000000,0.717274,0.979814,0.997466,1.058815,1.155671,0.818932,0.750826
4,0.0,0.458804,1.366424,0.061451,1.558624,-0.930649,1.306502,-0.472365,1.561282,2.173076,...,-0.772488,0.389387,0.000000,0.755844,0.884710,0.990756,1.647543,2.336182,1.857376,1.632091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.0,1.039675,-0.644821,-0.800909,0.573333,1.213948,0.781593,-1.508146,1.025201,2.173076,...,0.292693,-0.862045,0.000000,0.361094,1.208187,1.037595,0.683184,1.053465,0.807924,0.712753
999996,0.0,1.077924,-0.726635,-1.178769,1.371218,-0.472558,0.554865,0.458451,0.362168,0.000000,...,-1.787699,1.669014,0.000000,1.583988,1.166782,1.002019,1.327977,1.177066,0.965676,0.933131
999997,1.0,0.675121,1.271949,1.318212,0.914028,0.456076,0.967464,-2.446884,1.572924,0.000000,...,1.267929,-0.766600,0.000000,5.130123,3.183964,0.986169,1.473457,1.131897,2.264219,2.491497
999998,0.0,2.012002,-0.225041,1.132334,2.564906,0.825561,0.992564,-0.254515,-0.842595,0.000000,...,-1.015672,-0.797675,3.101961,1.268317,1.130358,0.994661,0.877500,0.729911,0.966467,1.288846


In [13]:
X_train = df_main.iloc[:800000, 1:]
Y_train = df_main.iloc[:800000, 0]
X_valid = df_main.iloc[800000:900000, 1:]
Y_valid = df_main.iloc[800000:900000, 0]
X_test = df_main.iloc[900000:, 1:]
Y_test = df_main.iloc[900000:, 0]

valid_set = set(X_valid, Y_valid)

In [29]:
model = Sequential()
model.add(Dense(units=300, activation='tanh', kernel_initializer=RandomNormal(mean=0.0, stddev=0.01)))
model.add(Dense(units=300, activation='tanh', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
model.add(Dense(units=300, activation='tanh', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
model.add(Dense(units=300, activation='tanh', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
model.add(Dense(units=300, activation='tanh', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
model.add(Dense(units=1, activation='sigmoid', kernel_initializer=RandomNormal(mean=0.0, stddev=0.001)))

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
def scheduler(epoch, lr):
    if lr <= 1e-6:
        return lr
    else:
        return lr / 1.0000002


my_callbacks = [
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10)
]

In [ ]:
history = model.fit(X_train, Y_train, epochs=1000, batch_size=100, validation_data=valid_set, callbacks=my_callbacks)

In [ ]:
# Врахувати, що val_set в оригіналі 500,000
# Ще потрібно добавити збільшення моменту (і поки не збільшиться до 0.99 не переривати навчання (тобто мінімум 200 епох)) 